In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [2]:
nflx = pd.read_csv("NFLX_2.csv")
nflx

,Date,Open,High,Low,Close,Adj Close,Volume,SMA30,SMA8,EMA50,EMA12,K,D,RSI,Month,Year,Day,Day_Name
0,2018-03-19,315.799988,317.000000,307.339996,313.480011,313.480011,9925200,291.264666,320.023750,299.206963,314.170719,59.605911,67.868628,62.913629,3,2018,19,Monday
1,2018-03-20,313.260010,319.500000,312.799988,317.500000,317.500000,5991900,293.372666,320.086250,300.216405,314.685819,67.527072,65.510667,64.226588,3,2018,20,Tuesday
2,2018-03-21,316.350006,319.399994,314.510010,316.480011,316.480011,5263900,295.064667,318.216251,301.099756,314.963171,65.517241,64.216742,64.205607,3,2018,21,Wednesday
3,2018-03-22,313.070007,314.119995,305.660004,306.700012,306.700012,8063300,296.469334,316.391254,301.399407,313.686766,25.013757,52.686023,53.106117,3,2018,22,Thursday
4,2018-03-23,307.410004,309.369995,300.359985,300.940002,300.940002,9529900,298.164000,314.523754,301.375172,311.719007,1.725213,30.752070,41.505554,3,2018,23,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,2022-01-31,401.970001,427.700012,398.200012,427.140015,427.140015,20047500,527.323000,402.152504,539.286213,435.893139,39.251083,24.587551,29.680923,1,2022,31,Monday
976,2022-02-01,432.959991,458.480011,425.540009,457.130005,457.130005,22542300,523.003001,395.762504,536.064401,439.160349,54.805248,37.039918,36.337520,2,2022,1,Tuesday
977,2022-02-02,448.250000,451.980011,426.480011,429.480011,429.480011,14346000,517.527668,399.760006,531.884621,437.671066,41.208485,45.088272,33.694538,2,2022,2,Wednesday
978,2022-02-03,421.440002,429.260010,404.279999,405.600006,405.600006,9905200,510.883669,402.066257,526.932283,432.737057,28.965820,41.659851,33.107304,2,2022,3,Thursday


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
nflx['Close'] = scaler.fit_transform(nflx['Close'].values.reshape(-1, 1))

In [4]:
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequence = data[i:i+sequence_length]
        target = data[i+sequence_length]
        sequences.append((sequence, target))
    return sequences

sequence_length = 10  # You can adjust this based on your data and requirements
sequences = create_sequences(nflx['Close'], sequence_length)

In [5]:
from sklearn.model_selection import train_test_split

sequences = np.array(sequences)
X = sequences[:, 0]
y = sequences[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

C:\Users\mikol\AppData\Local\Temp\ipykernel_14980\2850010980.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences = np.array(sequences)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(50, activation='tanh', input_shape=(sequence_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')


X_train = np.array(X_train)
y_train = np.array(y_train)

# Now, you can train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Series).

In [ ]:
y_pred = model.predict(X_test)

y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)
